<a href="https://colab.research.google.com/github/gmarchetti2020/topic_extraction/blob/main/topic_extraction_with_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

PROJECT_ID = "<<insert id>>"  # @param {type:"string"}
LOCATION = "<<inser location>>"  # @param {type:"string"}

if "google.colab" in sys.modules:
    # Define project information
    PROJECT_ID = PROJECT_ID
    LOCATION = LOCATION

    # Initialize Vertex AI
    import vertexai
    vertexai.init(project=PROJECT_ID, location=LOCATION)

In [2]:
def get_topic_extraction_prompt(content):
    prompt = f"""Label the main topic or topics in the following text: {content}"""
    prompt = prompt + """1. Identify and list the primary topic or category or provide a short description of the main subject matter of the text.
      2. If there are subtopics or secondary themes mentioned in the text, list them as well. If the text discusses multiple topics, provide a list of these topics and describe their relevance.
      3. Consider the context and tone of the text to determine the most appropriate topics. Take into account keywords, phrases, or specific terms that relate to the topics.
      4. If any notable entities (people, places, brands, products, etc.) are mentioned in the text that play a role in the topics, mention them and their associations.
      5. If the text suggests any actions, decisions, or recommendations related to the identified topics, provide a brief summary of these insights.

      Ensure that your labeling is clear, concise, and reflects the most significant topics or categories found in the text.

      Here's the output schema:

      ```


      {
          "Topic": "",
          "Subtopics": [""],
          "Context": "",
          "NotableEntities": [],
          "Recommendations": ""
      }


      ```

      Do not respond with your own suggestions or recommendations or feedback."""
    return prompt


In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

def execute_prompt(prompt, max_output_tokens=8192):
  model = GenerativeModel("gemini-pro")
  responses = model.generate_content(
    prompt,
    generation_config={
        "max_output_tokens": max_output_tokens,
        "temperature": 0,
        "top_p": 1
    },
  stream=True,
  )

  final_response = []

  for response in responses:
      final_response.append(response.candidates[0].content.parts[0].text)

  return ".".join(final_response)

In [5]:
import re
import json

def extract_json(input_string):
    # Extract JSON within block ```


    matches = re.findall(r'```(.*?)```', input_string, re.DOTALL)

    if matches:
        # Join the matches into a single string
        json_content = ''.join(matches)

        # Remove periods
        json_content = re.sub(r'\.', '', json_content)

        return json_content
    else:
        print("No ``` block found.")
        return None

In [9]:
summary = """The Odyssey, an epic poem attributed to Homer, tells the story of Odysseus, king of Ithaca, and his long journey home after the Trojan War. The story is divided into three parts:

The Adventures of Telemachus: Odysseus' son, Telemachus, sets out to find news of his missing father. He travels to Pylos and Sparta, where he learns that Odysseus is alive but held captive by the nymph Calypso.

The Wanderings of Odysseus: Odysseus recounts his ten years of wandering. He encounters various mythical creatures and gods, including the Cyclops Polyphemus, the sorceress Circe, and the Sire

The Return of Odysseus: Odysseus finally returns to Ithaca, disguised as a beggar. He reveals his identity to his son and a few loyal servants, then, together, they kill the suitors who have been vying for Penelope's hand and squandering Odysseus' wealth.

The Odyssey is not just an adventure story but also a tale of homecoming, loyalty, and perseverance. It explores themes of identity, temptation, and the power of cunning and intelligence. The poem also offers insights into ancient Greek culture, values, and beliefs.
"""

In [11]:
topics = []
prompt = get_topic_extraction_prompt(summary)
response = execute_prompt(prompt)
extracted_json = extract_json(response)
if extracted_json != None:
   topics.append(extracted_json)

In [14]:
import json
print(json.dumps(topics, indent=4))

[
    "json\n{\n  \"Topic\": \"The Odyssey\",\n  \"Subtopics\": [\n    \"Adventures of Telemachus\",\n    \"Wanderings of Odysseus\",\n    \"Return of Odysseus\",\n    \"Themes of identity, temptation, and the power of cunning and intelligence\",\n    \"Insights into ancient Greek culture, values, and beliefs\"\n  ],\n  \"Context\": \"Epic poem attributed to Homer\",\n  \"NotableEntities\": [\n    \"Odysseus\",\n    \"Telemachus\",\n    \"Calypso\",\n    \"Cyclops Polyphemus\",\n    \"Circe\",\n    \"Sirens\",\n    \"Penelope\"\n  ],\n  \"Recommendations\": \"None\"\n}\n"
]
